# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import nltk
from nltk import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')
from sqlalchemy import create_engine

from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table(con=engine, table_name='Message')
X = df.message.values
#X = df[['message','original','genre']]
#Y = df.aid_related.values
Y = df.drop(['id','message','original','genre'], axis=1).values


In [3]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X

array(['Weather update - a cold front from Cuba that could pass over Haiti',
       'Is the Hurricane over or is it not over',
       'Looking for someone but no name', ...,
       "Proshika, operating in Cox's Bazar municipality and 5 other unions, Ramu and Chokoria, assessment, 5 kg rice, 1,5 kg lentils to 700 families.",
       'Some 2,000 women protesting against the conduct of the elections were teargassed as they tried to converge on the local electoral commission offices in the southern oil city of Port Harcourt.',
       'A radical shift in thinking came about as a result of this meeting, recognizing that HIV/AIDS is at the core of the humanitarian crisis and identifying the crisis itself as a function of the HIV/AIDS pandemic.'], dtype=object)

In [5]:
Y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    tokens = nltk.tokenize.word_tokenize(text.lower().strip())
    lemmatizer = WordNetLemmatizer()
    clean_tokens = [lemmatizer.lemmatize(tok) for tok in tokens]

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [23]:
pipeline = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize, ngram_range=(1,3))),
    ('tfidf', TfidfTransformer()),
    ('multi_clf', MultiOutputClassifier(RandomForestClassifier(random_state=199), n_jobs=-1))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=199)

In [25]:
X_train

array([ 'It works as a fertilizer enhancement, a composting additive, and even as an alternative to household cleaning products, according to EMRO.',
       'I need a job to feed my family. I am an electrician and a musician. I perform harmonization and orchestration in Mexico. I speak English and French. Let me know how I can help',
       'The report also warns that if the clan fighting extends to sorghum-producing areas in the south during this critical stage of crop harvest, it could further hamper harvesting and exert a negative impact on food security throughout Somalia.',
       ...,
       'HELP THE EARTHQUAKE VICTIMS IN HAITI http tinyurl.com yk3bspe links to many resources where you can make donations. Broke? donate time',
       'The low snowfall pattern and the snowfall differentiation among catchments are clearly reflected in the irrigated wheat production levels.',
       "Having fun at #Lexi 's ! Real food in real house with electrical lighting . Good times . I feel huma

In [26]:
y_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 1, 1]])

In [27]:
print(X_train.shape, y_train.shape)

(19662,) (19662, 36)


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [28]:
X_train.shape

(19662,)

In [29]:
X_train

array([ 'It works as a fertilizer enhancement, a composting additive, and even as an alternative to household cleaning products, according to EMRO.',
       'I need a job to feed my family. I am an electrician and a musician. I perform harmonization and orchestration in Mexico. I speak English and French. Let me know how I can help',
       'The report also warns that if the clan fighting extends to sorghum-producing areas in the south during this critical stage of crop harvest, it could further hamper harvesting and exert a negative impact on food security throughout Somalia.',
       ...,
       'HELP THE EARTHQUAKE VICTIMS IN HAITI http tinyurl.com yk3bspe links to many resources where you can make donations. Broke? donate time',
       'The low snowfall pattern and the snowfall differentiation among catchments are clearly reflected in the irrigated wheat production levels.',
       "Having fun at #Lexi 's ! Real food in real house with electrical lighting . Good times . I feel huma

In [14]:
# train classifier
pipeline.fit(X_train, y_train)

# predict on test data
y_pred = pipeline.predict(X_test)

In [15]:
print(X_test.shape)

(6554,)


In [16]:
print(y_pred.shape)
y_pred

(6554, 36)


array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [17]:
y_pred[:,0]

array([1, 0, 1, ..., 0, 1, 0])

In [18]:
y_test

array([[1, 0, 0, ..., 0, 1, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 1, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [19]:
for category_idx in range(y_pred.shape[1]):
    print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[df.columns[4+category_idx] + '-0',df.columns[4+category_idx] + '-1']))

             precision    recall  f1-score   support

  related-0       0.59      0.33      0.43      1565
  related-1       0.81      0.93      0.86      4941

avg / total       0.76      0.78      0.76      6506

             precision    recall  f1-score   support

  request-0       0.89      0.98      0.93      5420
  request-1       0.83      0.40      0.53      1134

avg / total       0.88      0.88      0.86      6554

             precision    recall  f1-score   support

    offer-0       1.00      1.00      1.00      6524
    offer-1       0.00      0.00      0.00        30

avg / total       0.99      1.00      0.99      6554

               precision    recall  f1-score   support

aid_related-0       0.73      0.87      0.79      3900
aid_related-1       0.73      0.51      0.60      2654

  avg / total       0.73      0.73      0.72      6554

                precision    recall  f1-score   support

medical_help-0       0.93      0.99      0.96      6038
medical_help-1     

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [20]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'multi_clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'multi_clf__estimator__bootstrap', 'multi_clf__estimator__class_weight', 'multi_clf__estimator__criterion', 'multi_clf__estimator__max_depth', 'multi_clf__estimator__max_features', 'multi_clf__estimator__max_leaf_nodes', 'multi_clf__estimator__min_impurity_decrease', 'multi_clf__estimator__min_impurity_split', 'multi_clf__estimator__min_samples_leaf', 'multi_clf__estimator__min_samples_split', 'multi_clf__estimator__min_weight_fraction_leaf', 'multi_clf__estimator__n_estimators', 'multi_clf__estimator__n_jobs', 'multi_clf__estimator

In [30]:
parameters = {
        #'multi_clf__estimator__n_estimators': [5,10,20],
        #'multi_clf__estimator__max_depth': [50, 100]
    }

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=10)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [31]:
# train classifier
cv.fit(X_train, y_train)

# predict on test data
y_pred = cv.predict(X_test)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV]  ................................................................
[CV] ...................... , score=0.22017088800732376, total= 2.7min
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.0min remaining:    0.0s


[CV] ...................... , score=0.22978333841928594, total= 2.4min
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  5.7min remaining:    0.0s


[CV] ...................... , score=0.22642660970399756, total= 2.5min


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  8.4min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  8.4min finished


In [32]:
cv.best_params_

{}

In [33]:
for category_idx in range(y_pred.shape[1]):
    print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[df.columns[4+category_idx] + '-0',df.columns[4+category_idx] + '-1']))

             precision    recall  f1-score   support

  related-0       0.55      0.32      0.41      1565
  related-1       0.81      0.91      0.86      4941

avg / total       0.74      0.77      0.75      6506

             precision    recall  f1-score   support

  request-0       0.89      0.98      0.93      5420
  request-1       0.81      0.39      0.53      1134

avg / total       0.87      0.88      0.86      6554

             precision    recall  f1-score   support

    offer-0       1.00      1.00      1.00      6524
    offer-1       0.00      0.00      0.00        30

avg / total       0.99      1.00      0.99      6554

               precision    recall  f1-score   support

aid_related-0       0.71      0.89      0.79      3900
aid_related-1       0.75      0.46      0.57      2654

  avg / total       0.73      0.72      0.70      6554

                precision    recall  f1-score   support

medical_help-0       0.93      1.00      0.96      6038
medical_help-1     

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [122]:
# Output a pickle file for the model
joblib.dump(cv, 'initial_model.pkl') 

['initial_model.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.